<h1>Inteligência Aritifical - Algoritmo Genético - Problemas das 8 Rainhas <h1>

<h1> Aluno: Brainer Sueverti de Campos <h1>
<h1> Ra: 790829 <h1>

Bibliotecas necessárias:

random - para gerar números aleatórios
pandas - para realizar as análises

In [1]:
import random
import pandas as pd

<h2> Primeiramente serão definidas todas as funções necessárias para a solução do problema. <h2>

A função abaixo gera os cromossomos aleatóriamente utilizando a função random. Cada posição do vetor (cromossomo) está a posição da rainha na coluna.

In [2]:
def cromossomos_aleatorios(n):
    return [random.randint(0, n - 1) for _ in range(n)]

In [3]:
def pop_aletoria():
    x = []
    for i in range(0,pop_max):
        x.append(cromossomos_aleatorios(num_queen))
    return x

<h3> A função fitness abaixo retorna a quantidade de pares de rainhas que não estão em colisão. Para o caso n=8, o número ideal de fitness é n*(n-1)/2 = 28. Primeiramente é calculada as coliões horizontais e depois as colisões diagonais, que subtrai do fitness ideal<h3>

In [4]:
def fitness(cromossomo):

    max_fit = (len(cromossomo) * (len(cromossomo) - 1)) / 2

    colisoes_horizontais = (
        sum([cromossomo.count(queen) - 1 for queen in cromossomo]) / 2
    )
    n = len(cromossomo)
    esq_diagonal = [0] * (2 * n - 1)
    dir_diagonal = [0] * (2 * n - 1)
    for i in range(n):
        esq_diagonal[i + cromossomo[i] - 1] += 1
        dir_diagonal[len(cromossomo) - i + cromossomo[i] - 2] += 1

    colisoes_diagonais = 0
    for i in range(2 * n - 1):
        counter = 0
        if esq_diagonal[i] > 1:
            counter += esq_diagonal[i] - 1
        if dir_diagonal[i] > 1:
            counter += dir_diagonal[i] - 1
        colisoes_diagonais += counter

    return int(max_fit - (colisoes_horizontais + colisoes_diagonais))

In [5]:
def all_fitness(populacao):
  
  fit = []
  for i in range(len(populacao)):
    f = fitness(populacao[i])
    fit.append(f)

  return fit

<h1> 
A função abaixo realiza o cruzamento entre dois cromossomos, utilizando como resultado a primeira metade de um cromossomo x como sua primeira metade e a segunda metade como a segunda metade de um cromossomo y.

Exemplo:  

x = [1,2,3,4,1,2,3,4]

y = [5,6,7,8,5,6,7,8]

resultado = [1,2,3,4,5,6,7,8]
<h1>

In [6]:
def crossover(x,y):
    #tamanho do cromossomo
    n = len(x)
    #criação do resultado do crossover
    resultado1 = [0] * n
    resultado2 = [0] * n
    #crossover
    #a primeira metade do resultado é a primeira metade de x
    #a segunda metade do resultado é a segunda metade de y
    for i in range(n):
        if i<int(n/2):
            resultado1[i] = x[i]
            resultado2[i] = y[i]
        else:
            resultado1[i] = y[i]
            resultado2[i] = x[i]
    return resultado1,resultado2

<h1> função abaxio realiza a mutação em determinada taxa, na qual é selecionada uma posição e mudado o valor. <h1>

In [2]:
def mutacao(filhos,taxa):

  for i in range(len(filhos)):
    if random.random() < taxa:
      pos = random.randint(0,num_queen-1)
      filhos[i][pos] = random.randint(0,num_queen-1)

  return filhos

<h4> A seleção dos cromossomos que serão cruzados se dá através da função abaixo. É sorteado dois números aleatórios como posição do vetor fitnesses, que armazenam os valores da função fitness para cada cromossomo da população, utilizando o valor da posição é possível encontrar os cromossomos e, assim, dentre esses dois cromossomos é feito a escolha do que tiver o maior valor da função fitness, isso é feito para selecionar dois cromossomos. <h4>  

In [8]:
def selecao_torneio(fitnesses):
  
  ind1 = -1
  ind2 = -1

  while ind1 == ind2:
    # Torneio 1
    sorteados = random.sample(range(0,pop_max), 2)
    if fitnesses[sorteados[0]] > fitnesses[sorteados[1]]:
      ind1 = sorteados[0]
    else:
      ind1 = sorteados[1]

    # Torneio 2
    sorteados = random.sample(range(0,pop_max), 2)
    if fitnesses[sorteados[0]] > fitnesses[sorteados[1]]:
      ind2 = sorteados[0]
    else:
      ind2 = sorteados[1]  
        
  return ind1,ind2

<h5> A função abaixo seleciona os dois maiores fitness da população <h5>

In [9]:
def elitismo(fitnesses):

  x = fitnesses.index(max(fitnesses))
  fitnesses[x] = 0
  y = fitnesses.index(max(fitnesses))

  return x,y

<h1>Abaixo é implementado o algoritmo genético com a funções criadas acima. O algoritmo segue os passos abaixo:<h1>
<h2>(1) Gerada uma população alaeatória

    (2) Calculado o fitness de todos os cromossomos
    
    (3) Verificado se há o fitness máximo (solução)

    (4) Seleciona os cromossomos mais promissores (fitness maior) para a nova população
    
    (5) 
<h2>

In [10]:
num_queen = 8
pop_max = 1000
pop_inicial = pop_aletoria()
ger_atual = 0
ger_max = 1000
encontrou = False
while(ger_atual != ger_max and encontrou != True):
    nova_pop = [[0 for i in range(num_queen)] for i in range(pop_max)]
    fitnesses = all_fitness(pop_inicial)
    for i in range(0,len(fitnesses)):
        if fitnesses[i]==28:
            encontrou = True
            solucao = pop_inicial[i]
            ger_solucao = ger_atual
            print("A solução foi encontrada:",solucao,"na geração ",ger_atual)
            break
        elif fitnesses[i]==27:
            solucao = pop_inicial[i]
    elite = elitismo(fitnesses.copy()) # Copiando por causa da referencia do vetor
    nova_pop[0] = pop_inicial[elite[0]]
    nova_pop[1] = pop_inicial[elite[1]]
    num_filhos = 2
    while num_filhos < pop_max:
        ind_vencedores = selecao_torneio(fitnesses)
        filhos = crossover(pop_inicial[ind_vencedores[0]],pop_inicial[ind_vencedores[1]])
        filhos = mutacao(filhos,0.5)
        nova_pop[num_filhos] = filhos[0]
        nova_pop[num_filhos+1] = filhos[1]
        num_filhos = num_filhos + 2
    pop_inicial = nova_pop.copy()
    nova_pop.clear()
    ger_atual = ger_atual + 1
if(ger_atual==ger_max):
    print("A solução ótima não foi encontrada,a melhor solução encontrada foi: ",solucao)


A solução ótima não foi encontrada,a melhor solução encontrada foi:  [4, 0, 7, 5, 2, 0, 6, 3]
